# A/B Test: A New Menu Launch
## Project Overview
You're a business analyst for Round Roasters, a coffee restaurant in the United States of America. The executive team conducted a market test with a new menu and needs to figure whether the new menu can drive enough sales to offset the cost of marketing the new menu. Your job is to analyze the A/B test and write up a recommendation to whether the Round Roasters chain should launch this new menu.

In [ ]:
# Load package
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.neighbors import KDTree
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = [12, 12]

## Step 1: Plan Your Analysis
To perform the correct analysis, you will need to prepare a data set. Prior to rolling up your sleeves and preparing the data, it’s a good idea to have a plan of what you need to do in order to prepare the correct data set. A good plan will help you with your analysis. Here are a few questions to get you started:

-What is the performance metric you’ll use to evaluate the results of your test?  
-What is the test period?  
-At what level (day, week, month, etc.) should the data be aggregated?  

In [ ]:
# load Stores data
stores_data = pd.read_csv('round-roaster-stores.csv')
stores_data.info()

In [ ]:
stores_data.head(3)

In [ ]:
# load Transactions data
# force Invoice Date column to datetime 
transactions_data = pd.read_csv('RoundRoastersTransactions.csv', parse_dates=['Invoice Date'])
transactions_data.info()

In [ ]:
transactions_data.head()

In [ ]:
# load Treatment Stores data
treatment_stores_data = pd.read_csv('treatment-stores.csv')
treatment_stores_data.info()

In [ ]:
treatment_stores_data.head(3)

## Step 2: Clean Up Your Data
In this step, you should prepare the data for steps 3 and 4. You should aggregate the transaction data to the appropriate level and filter on the appropriate data ranges. You can assume that there is no missing, incomplete, duplicate, or dirty data. You’re ready to move on to the next step when you have weekly transaction data for all stores.

In [ ]:
# Test cities: Denver and Chicago
# Treatment: 12 Weeks [2016-April-29 to 2016-July-21], start on Friday
# Control: 12 Weeks [(2015-April-29 to 2015-July-21], start on Wednesday
# Total weeks to identify trend and season: 52 Weeks + treatment weeks + control weeks = 52 + 12 + 12 = 76 Weeks

data_end_date = datetime(2016, 7, 21)
data_start_date = data_end_date - timedelta(weeks=76)
print(f'Data Start Date: {data_start_date} \nData End Date: {data_end_date}')

In [ ]:
# Filter data for further process
filtered_transactions_data = transactions_data.query('`Invoice Date` > @data_start_date and `Invoice Date` <= @data_end_date')
# Make Invoice Date Index
filtered_transactions_data.set_index('Invoice Date', inplace=True)

filtered_transactions_data.head(10)

In [ ]:
# Aggregate the data to get the weekly gross margin and weekly traffic count unique invoices
weekly_gross_and_traffic = filtered_transactions_data.groupby([pd.Grouper(freq='W-FRI', closed='left'), 'StoreID'], as_index=True).agg({'Gross Margin': 'sum', 'Invoice Number': 'nunique'}).reset_index()
weekly_gross_and_traffic.rename(columns={'Invoice Number': 'Weekly Foot Traffic'}, inplace=True)

# Hack to start week on first date
weekly_gross_and_traffic['Invoice Date'] = weekly_gross_and_traffic['Invoice Date'] - pd.offsets.Week(1)
weekly_gross_and_traffic.head()

In [ ]:
# How many transactions did you get for Store 10018 in the week starting 2015-02-06?
weekly_gross_and_traffic.query('StoreID == 10018 and `Invoice Date` == @datetime(2015, 2, 6)')

In [ ]:
# filtered_transactions_data per store
temp_df = weekly_gross_and_traffic.groupby(['StoreID'], as_index=False).sum()

# join stores and transactions  per store
stores_columns = ['StoreID', 'Sq_Ft', 'AvgMonthSales', 'Region']
stores_transactions_merged = temp_df.merge(stores_data[stores_columns], on='StoreID')
stores_transactions_merged.head()

In [ ]:
# Correlation between control variable and perfromance metric
stores_transactions_merged.drop(columns='StoreID').corr()

In [ ]:
# correlated variables to be used in KDTree
control_variables = ['Weekly Foot Traffic', 'AvgMonthSales', 'Gross Margin']

In [ ]:
# filter weekly data
weekly_data = weekly_gross_and_traffic.groupby(['Invoice Date'], as_index=False).sum().drop(columns=['StoreID', 'Gross Margin']).set_index('Invoice Date')
weekly_data.head()

In [ ]:
res = seasonal_decompose(weekly_data, period=12, extrapolate_trend='freq')
# res.plot()
# res.trend
# res.seasonal

In [ ]:
stores_transactions_merged['Region'].value_counts()

## Step 3: Match Treatment and Control Units
In this step, you should create the trend and seasonality variables, and use them along with you other control variable(s) to match two control units to each treatment unit. Treatment stores should be matched to control stores in the same region. Note: Calculate the number of transactions per store per week and use 12 periods to calculate trend and seasonality.  

Apart from trend and seasonality...  

-What control variables should be considered? Note: Only consider variables in the RoundRoastersStore file.  
-What is the correlation between your each potential control variable and your performance metric? (Example of correlation matrix below)  
-What control variables will you use to match treatment and control stores?

In [ ]:
# region data control
west_region_control = stores_transactions_merged.query('Region == "West"').reset_index().drop(columns='index')
central_region_control = stores_transactions_merged.query('Region == "Central"').reset_index().drop(columns='index')

print(central_region_control.info())

# region data treatment
west_region_treatment = treatment_stores_data.query('Region == "West"')['StoreID'].values
central_region_treatment = treatment_stores_data.query('Region == "Central"')['StoreID'].values
print(west_region_treatment)

In [ ]:
# Scale the features
transformer = ColumnTransformer([('scaler', StandardScaler(), control_variables)], remainder='drop')
scaled_west_region_control = transformer.fit_transform(west_region_control)
scaled_central_region_control = transformer.fit_transform(central_region_control)
# KDtree for west
west_kdtree = KDTree(scaled_west_region_control, leaf_size=2)
# KDtree for central
central_kdtree = KDTree(scaled_central_region_control, leaf_size=2)

## Step 4: Analysis and Writeup
Conduct your A/B analysis and create a short report outlining your results and recommendations.  

In an AB Analysis we use the correlation matrix to find the most correlated variable to the performance metric to include in the AB controls tool to help find the best matches.